In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Conv2D,Dense,Flatten,BatchNormalization,MaxPool2D
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import cv2

In [2]:
tf.__version__

'2.0.0'

In [3]:
def make_model():
    input_ = Input((28,28,1))
    x = Conv2D(32,5,padding="same",activation="relu")(input_)
    x = BatchNormalization()(x)
    x = MaxPool2D(padding="same")(x)
    x = Conv2D(64,5,padding="same",activation="relu")(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(padding="same")(x)
    x = Conv2D(128,5,padding="same",activation="relu")(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(padding="same")(x)
    x = Flatten()(x)
    x = Dense(10,"softmax")(x)
    model = Model(input_,x)
    return model

In [4]:
model = make_model()

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0     

In [5]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()

In [7]:
def make_dataset(X,y):
    
    def preprocess(X,y):
        #return tf.expand_dims((X - 127.5)/128,3),tf.expand_dims(y,1)
        X = tf.cast(X,"float32")
        y = tf.cast(y,"float32")
        return tf.expand_dims((X - 127.5)/128,2),tf.expand_dims(y,0)
    dataset = tf.data.Dataset.from_tensor_slices((X,y))
    dataset = dataset.map(preprocess)
    
    return dataset
train_dataset = make_dataset(X_train,y_train)
train_dataset

<MapDataset shapes: ((28, 28, 1), (1,)), types: (tf.float32, tf.float32)>

In [16]:
step = 1
train_dataset = make_dataset(X_train,y_train)
train_dataset = train_dataset.shuffle(buffer_size=1000,reshuffle_each_iteration=True)
train_dataset = train_dataset.batch(128)

optim = tf.optimizers.Adam()
loss_fun = tf.losses.SparseCategoricalCrossentropy()
acc = tf.metrics.SparseCategoricalAccuracy()

@tf.function
def train_on_batch(X,y):
    with tf.GradientTape() as tape:
        pred = model(X)
        loss_val = loss_fun(y,pred)
    gradients = tape.gradient(loss_val, model.trainable_weights)
    optim.apply_gradients(zip(gradients, model.trainable_weights))
    acc.update_state(y,pred)
    return loss_val

with tqdm.tqdm(train_dataset) as pbar:
    acc.reset_states()
    for i,(X,y) in enumerate(pbar):
        loss = train_on_batch(X,y)
        pbar.set_postfix({
            "loss":loss.numpy(),
            "acc":acc.result().numpy()
        })

469it [02:10,  3.59it/s, loss=0.0507, acc=0.983]


In [17]:
model.save("model/tiny_mnist.h5")

In [25]:
cv2.imwrite("mnist1.png",X_train[0])

True